## ДЗ_5

### Тема «POS-tagger и NER»

**Задание 1. Написать теггер на данных с русским языком.**

1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации.
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов.
3. сравнить все реализованные методы, сделать выводы.


**Задание 2. Проверить, насколько хорошо работает NER.**

1. проверить NER из nltk/spacy/deeppavlov.

2. написать свой NER, попробовать разные подходы.

*a. передаём в сетку токен и его соседей.*

*b. передаём в сетку только токен.*

*c. свой вариант.*

3. сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.

### Задание 1.

**Загрузим необходимые библиотеки и данные.**

In [2]:
!pip install pyconll

In [3]:
!wget -O ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2022-10-25 12:41:21--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 2606:50c0:8000::154, 2606:50c0:8003::154, 2606:50c0:8002::154, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 40736565 (39M) [text/plain]
Сохранение в: «ru_syntagrus-ud-train.conllu»

ru_syntagrus-ud-tra 100%[===================>]  38,85M  15,1MB/s    за 2,6s    

2022-10-25 12:41:27 (15,1 MB/s) - «ru_syntagrus-ud-train.conllu» сохранён [40736565/40736565]

--2022-10-25 12:41:27--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Подключение к raw.githubuserco

In [4]:
import pandas as pd
import numpy as np
import re

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import pyconll
import nltk
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger, RegexpTagger

import warnings
warnings.filterwarnings("ignore")

In [5]:
data_train = pyconll.load_from_file('ru_syntagrus-ud-train.conllu')
data_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [6]:
fdata_train = []
for sent in data_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in data_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in data_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [7]:

len(fdata_train), len(fdata_test), len(fdata_sent_test)

(24516, 8906, 8906)

In [16]:

fdata_train[:2]

[[('Анкета', 'NOUN'), ('.', 'PUNCT')],
 [('Начальник', 'NOUN'),
  ('областного', 'ADJ'),
  ('управления', 'NOUN'),
  ('связи', 'NOUN'),
  ('Семен', 'PROPN'),
  ('Еремеевич', 'PROPN'),
  ('был', 'AUX'),
  ('человек', 'NOUN'),
  ('простой', 'ADJ'),
  (',', 'PUNCT'),
  ('приходил', 'VERB'),
  ('на', 'ADP'),
  ('работу', 'NOUN'),
  ('всегда', 'ADV'),
  ('вовремя', 'ADV'),
  (',', 'PUNCT'),
  ('здоровался', 'VERB'),
  ('с', 'ADP'),
  ('секретаршей', 'NOUN'),
  ('за', 'ADP'),
  ('руку', 'NOUN'),
  ('и', 'CCONJ'),
  ('иногда', 'ADV'),
  ('даже', 'PART'),
  ('писал', 'VERB'),
  ('в', 'ADP'),
  ('стенгазету', 'NOUN'),
  ('заметки', 'NOUN'),
  ('под', 'ADP'),
  ('псевдонимом', 'NOUN'),
  ('"', 'PUNCT'),
  ('Муха', 'NOUN'),
  ('"', 'PUNCT'),
  ('.', 'PUNCT')]]

**Проверим работу всех теггеров поочередно.**

In [25]:
default_tagger = nltk.DefaultTagger('NN')
default_acc = default_tagger.evaluate(fdata_test)

unigram_tagger = UnigramTagger(fdata_train)
unigram_acc = unigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train)
bigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train)
trigram_acc = trigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_unigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_bigram_unigram_acc = trigram_tagger.evaluate(fdata_test)

print(f'Accuracy:\nDefault Tagger: {round(default_acc, 3)},\nUnigram Tagger: {round(unigram_acc, 3)},\nBigram Tagger: {round(bigram_acc, 5)},\n'
      f'Trigram Tagger: {round(trigram_acc, 3)},\nBigram and Unigram Tagger: {round(bigram_unigram_acc, 5)},\n'
      f'Trigram, Bigram and Unigram Tagger: {round(trigram_bigram_unigram_acc, 5)},\n')

Accuracy:
Default Tagger: 0.0,
Unigram Tagger: 0.824,
Bigram Tagger: 0.60939,
Trigram Tagger: 0.178,
Bigram and Unigram Tagger: 0.82928,
Trigram, Bigram and Unigram Tagger: 0.82914,



**Различные комбинации теггеров могут давать прирост качества.**

**Для эксперимента попробуем объединить работу всех теггеров с помощью функции.**

In [31]:
def union_taggers(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = union_taggers(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(fdata_test) 

0.827905462595221

**Получили некий усредненный результат. Не самый высокий.**
**ВЫВОД: на каждом корпусе пробовать все варианты и выбирать наилучший.**

### Попробуем написать теггер.

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder

**Преобразуем тренировочный датасет в списки слов и списки POS-разметки.**

In [33]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(' ' if tok[0] is None else tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [35]:
train_tok[:7], train_label[:7]

(['Анкета', '.', 'Начальник', 'областного', 'управления', 'связи', 'Семен'],
 ['NOUN', 'PUNCT', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'PROPN'])

In [37]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
train_enc_labels

array([ 7, 13,  7, ...,  1, 11, 13])

In [38]:
test_enc_labels = le.transform(test_label)
test_enc_labels

array([ 7, 13,  1, ...,  0,  7, 13])

In [41]:
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [43]:
%time

vectorizers = [CountVectorizer(ngram_range=(1, 5), analyzer='char'), 
               TfidfVectorizer(ngram_range=(1, 5), analyzer='char'), 
               HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=1000)] 
vectorizers_word = [CountVectorizer(ngram_range=(1, 5), analyzer='word'), 
               TfidfVectorizer(ngram_range=(1, 5), analyzer='word'), 
               HashingVectorizer(ngram_range=(1, 5), analyzer='word', n_features=1000)] 
n_features = [2000, 3000, 5000]
vectorizers_hash = [HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=feat) for feat in n_features]
vectorizers_hash_word = [HashingVectorizer(ngram_range=(1, 5), analyzer='word', n_features=feat) for feat in n_features]
f1_scores = []
accuracy_scores = []

for vectorizer in vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word:
    X_train = vectorizer.fit_transform(train_tok)
    X_test = vectorizer.transform(test_tok)
    
    lr = LogisticRegression(random_state=0, max_iter=100)
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)
    f1 = f1_score(test_enc_labels, pred, average='weighted')
    f1_scores.append(f1)
    acc = accuracy_score(test_enc_labels, pred)
    accuracy_scores.append(acc)
    
    print(vectorizer)
    print(classification_report(test_enc_labels, pred, target_names=le.classes_))

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.05 µs
CountVectorizer(analyzer='char', ngram_range=(1, 5))
              precision    recall  f1-score   support

         ADJ       0.94      0.92      0.93     15103
         ADP       0.98      1.00      0.99     13717
         ADV       0.91      0.93      0.92      7783
         AUX       0.82      0.96      0.88      1390
       CCONJ       0.89      0.97      0.93      5672
         DET       0.83      0.79      0.81      4265
        INTJ       0.39      0.29      0.33        24
        NOUN       0.94      0.97      0.96     36238
      NO_TAG       1.00      0.77      0.87       265
         NUM       0.86      0.89      0.88      1734
        PART       0.95      0.77      0.85      5125
        PRON       0.90      0.84      0.87      7444
       PROPN       0.84      0.66      0.73      5473
       PUNCT       1.00      1.00      1.00     29186
       SCONJ       0.75      0.97      0.85      2865
         SYM  

**Для удобства представим данные в виде таблицы:**

In [45]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [46]:
result_model = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'f1_score': f1_scores})
result_model.sort_values('f1_score', ascending=False)

,Vectorizer,f1_score
0,"CountVectorizer(analyzer='char', ngram_range=(1, 5))",0.938119
1,"TfidfVectorizer(analyzer='char', ngram_range=(1, 5))",0.927187
8,"HashingVectorizer(analyzer='char', n_features=5000, ngram_range=(1, 5))",0.909308
7,"HashingVectorizer(analyzer='char', n_features=3000, ngram_range=(1, 5))",0.898586
6,"HashingVectorizer(analyzer='char', n_features=2000, ngram_range=(1, 5))",0.892840
2,"HashingVectorizer(analyzer='char', n_features=1000, ngram_range=(1, 5))",0.866751
4,"TfidfVectorizer(ngram_range=(1, 5))",0.650066
3,"CountVectorizer(ngram_range=(1, 5))",0.644079
11,"HashingVectorizer(n_features=5000, ngram_range=(1, 5))",0.581934
10,"HashingVectorizer(n_features=3000, ngram_range=(1, 5))",0.568540


In [47]:

result_model_acc = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'Accuracy': accuracy_scores})
result_model_acc.sort_values('Accuracy', ascending=False)

,Vectorizer,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(1, 5))",0.939469
1,"TfidfVectorizer(analyzer='char', ngram_range=(1, 5))",0.929390
8,"HashingVectorizer(analyzer='char', n_features=5000, ngram_range=(1, 5))",0.912540
7,"HashingVectorizer(analyzer='char', n_features=3000, ngram_range=(1, 5))",0.902110
6,"HashingVectorizer(analyzer='char', n_features=2000, ngram_range=(1, 5))",0.896243
2,"HashingVectorizer(analyzer='char', n_features=1000, ngram_range=(1, 5))",0.870571
4,"TfidfVectorizer(ngram_range=(1, 5))",0.639729
3,"CountVectorizer(ngram_range=(1, 5))",0.634520
11,"HashingVectorizer(n_features=5000, ngram_range=(1, 5))",0.598301
10,"HashingVectorizer(n_features=3000, ngram_range=(1, 5))",0.592408


### В результате эксперимента проверили различные векторайзеры для разного количества символов. Наилучший результат показали символьные N-граммы. Результат N-грамм для слов оказался значительно хуже.

### Задание 2.

In [50]:
!pip install corus

In [52]:
!pip install razdel
!pip install -U spacy
!python -m spacy info

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 7.9 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 3.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.4/754.4 kB 5.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 1.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 402.8 kB/s eta 0:00:00a 0:00:01
  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 12.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.2.0
    Uninstalling smart-open-6.2.0:
      Successfully uninstalled smart-open-6.2.0
2022-10-25 14:46:38.086876: I tensorflow/core/platform/cpu_feature_

### NLTK

In [56]:
import nltk
from nltk.tokenize import word_tokenize
import matplotlib
import pyconll
import corus
from corus import load_ne5
import re
import spacy
from spacy import displacy

In [78]:
import nltk
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package tagsets to /Users/mac/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mac/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/mac/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/mac/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [59]:
nltk.help.upenn_tagset('RB')
nltk.help.upenn_tagset('NN')
nltk.help.upenn_tagset('VB')

RB: adverb
    occasionally unabatingly maddeningly adventurously professedly
    stirringly prominently technologically magisterially predominately
    swiftly fiscally pitilessly ...
NN: noun, common, singular or mass
    common-carrier cabbage knuckle-duster Casino afghan shed thermostat
    investment slide humour falloff slick wind hyena override subhumanity
    machinist ...
VB: verb, base form
    ask assemble assess assign assume atone attention avoid bake balkanize
    bank begin behold believe bend benefit bevel beware bless boil bomb
    boost brace break bring broil brush build ...


In [60]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2022-10-25 16:06:23--  http://www.labinform.ru/pub/named_entities/collection5.zip
Распознаётся www.labinform.ru (www.labinform.ru)… 95.181.230.181
Подключение к www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 1899530 (1,8M) [application/zip]
Сохранение в: «collection5.zip»

collection5.zip     100%[===================>]   1,81M  10,6MB/s    за 0,2s    

2022-10-25 16:06:23 (10,6 MB/s) - «collection5.zip» сохранён [1899530/1899530]



In [61]:
!unzip collection5.zip

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection5/006.txt     
  inflating: Collection5/007.ann     
  inflating: Collection5/007.txt     
  inflating: Collection5/008.ann     
  inflating: Collection5/008.txt     
  inflating: Collection5/009.ann     
  inflating: Collection5/009.txt     
  inflating: Collection5/010.ann     
  inflating: Collection5/010.txt     
  inflating: Collection5/011.ann     
  inflating: Collection5/011.txt     
  inflating: Collection5/012.ann     
  inflating: Collection5/012.txt     
  inflating: Collection5/013.ann    

In [71]:
records = load_ne5('Collection5/')

In [72]:
document = next(records)
text = document.text
# text
text = re.sub('\r\n\r\n',' ',text)
text = re.sub('\r\n',' ',text)
text

'Жириновский предлагает обменять с США Сноудена на Бута Лидер ЛДПР Владимир Жириновский предложил обменять бывшего сотрудника ЦРУ США Эдварда Сноудена, который прибыл в Москву, на осужденного в Америке бизнесмена Виктора Бута. "Сноудена ни в коем случае не высылать в США, а обменять на Виктора Бута и Константина Ярошенко. В идеале — добавить генерала Олега Калугина", — написал он в своем микроблоге в Twitter. Сноуден, работавший на компанию Booz Allen Hamilton — подрядчика Центрального разведывательного управления США, в начале июня распространил секретный ордер суда, по которому спецслужбы получили доступ ко всем звонкам крупнейшего сотового оператора Verizon, а также данные о сверхсекретной программе агентства национальной безопасности PRISM, позволяющей отслеживать электронные коммуникации на крупнейших сайтах. В воскресенье стало известно, что Сноуден прибыл из Гонконга в Москву и запросил убежища в Эквадоре. Что ждет Эдварда Сноудена Эдвард Сноуден, наверное, не знал только одного

In [73]:
document.text = text

In [74]:
nltk.pos_tag(nltk.word_tokenize(text)) # Как предварительно очистить все статьи в словаре.

[('Жириновский', 'JJ'),
 ('предлагает', 'NNP'),
 ('обменять', 'NNP'),
 ('с', 'NNP'),
 ('США', 'NNP'),
 ('Сноудена', 'NNP'),
 ('на', 'NNP'),
 ('Бута', 'NNP'),
 ('Лидер', 'NNP'),
 ('ЛДПР', 'NNP'),
 ('Владимир', 'NNP'),
 ('Жириновский', 'NNP'),
 ('предложил', 'NNP'),
 ('обменять', 'NNP'),
 ('бывшего', 'NNP'),
 ('сотрудника', 'NNP'),
 ('ЦРУ', 'NNP'),
 ('США', 'NNP'),
 ('Эдварда', 'NNP'),
 ('Сноудена', 'NNP'),
 (',', ','),
 ('который', 'NNP'),
 ('прибыл', 'NNP'),
 ('в', 'NNP'),
 ('Москву', 'NNP'),
 (',', ','),
 ('на', 'NNP'),
 ('осужденного', 'NNP'),
 ('в', 'NNP'),
 ('Америке', 'NNP'),
 ('бизнесмена', 'NNP'),
 ('Виктора', 'NNP'),
 ('Бута', 'NNP'),
 ('.', '.'),
 ('``', '``'),
 ('Сноудена', 'JJ'),
 ('ни', 'NN'),
 ('в', 'NNP'),
 ('коем', 'NNP'),
 ('случае', 'NNP'),
 ('не', 'NNP'),
 ('высылать', 'NNP'),
 ('в', 'NNP'),
 ('США', 'NNP'),
 (',', ','),
 ('а', 'NNP'),
 ('обменять', 'NNP'),
 ('на', 'NNP'),
 ('Виктора', 'NNP'),
 ('Бута', 'NNP'),
 ('и', 'NNP'),
 ('Константина', 'NNP'),
 ('Ярошенко', 'NN

In [79]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))) if hasattr(chunk, 'label') }

{('Hamilton', 'PERSON'),
 ('Америке', 'PERSON'),
 ('Виктора Бута', 'PERSON'),
 ('Москву', 'PERSON'),
 ('Сноуден', 'PERSON'),
 ('Эдварда Сноудена', 'PERSON'),
 ('Эдварда Сноудена Эдвард Сноуден', 'PERSON')}

**В целом - работает. Но ошибки встречаются ('Америке', 'PERSON').**

In [80]:
document.spans

[Ne5Span(
     index='T1',
     type='PER',
     start=0,
     stop=11,
     text='Жириновский'
 ),
 Ne5Span(
     index='T2',
     type='GEOPOLIT',
     start=34,
     stop=37,
     text='США'
 ),
 Ne5Span(
     index='T3',
     type='PER',
     start=38,
     stop=46,
     text='Сноудена'
 ),
 Ne5Span(
     index='T4',
     type='PER',
     start=50,
     stop=54,
     text='Бута'
 ),
 Ne5Span(
     index='T5',
     type='ORG',
     start=64,
     stop=68,
     text='ЛДПР'
 ),
 Ne5Span(
     index='T6',
     type='PER',
     start=69,
     stop=89,
     text='Владимир Жириновский'
 ),
 Ne5Span(
     index='T7',
     type='ORG',
     start=128,
     stop=131,
     text='ЦРУ'
 ),
 Ne5Span(
     index='T8',
     type='GEOPOLIT',
     start=132,
     stop=135,
     text='США'
 ),
 Ne5Span(
     index='T9',
     type='PER',
     start=136,
     stop=152,
     text='Эдварда Сноудена'
 ),
 Ne5Span(
     index='T10',
     type='LOC',
     start=171,
     stop=177,
     text='Москву'
 ),
 Ne5

### SpaCy

In [81]:
!python -m spacy download ru_core_news_sm

2022-10-25 16:26:02.950553: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 14.1 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [86]:
import spacy
from spacy import displacy
import ru_core_news_sm
from spacy.lang.ru.examples import sentences 
from spacy.lang.ru import Russian

In [85]:
nlp = spacy.load("ru_core_news_sm")

In [88]:
ny_bb = text
article = nlp(ny_bb)

In [89]:
displacy.render(article, jupyter=True, style='ent')

**На этом тексте библиотека SpaCy без ошибок.**

**Посмотрим на список токенов, частей речи и сущностей.**

In [90]:
for token in article:
    print(token.text, token.pos_, token.dep_)

Жириновский PROPN nsubj
предлагает VERB ROOT
обменять VERB xcomp
с ADP case
США PROPN obl
Сноудена PROPN obj
на ADP case
Бута PROPN obl
Лидер NOUN nsubj
ЛДПР PROPN nmod
Владимир PROPN appos
Жириновский PROPN flat:name
предложил VERB conj
обменять VERB xcomp
бывшего ADJ amod
сотрудника NOUN obj
ЦРУ PROPN nmod
США PROPN nmod
Эдварда PROPN appos
Сноудена PROPN flat:name
, PUNCT punct
который PRON nsubj
прибыл VERB acl:relcl
в ADP case
Москву PROPN obl
, PUNCT punct
на ADP case
осужденного NOUN acl
в ADP case
Америке PROPN obl
бизнесмена NOUN appos
Виктора PROPN appos
Бута PROPN flat:name
. PUNCT punct
" PUNCT punct
Сноудена NOUN obj
ни PART advmod
в ADP fixed
коем DET fixed
случае NOUN fixed
не PART advmod
высылать VERB ROOT
в ADP case
США PROPN obl
, PUNCT punct
а CCONJ cc
обменять VERB conj
на ADP case
Виктора PROPN obl
Бута PROPN flat:name
и CCONJ cc
Константина PROPN conj
Ярошенко PROPN flat:name
. PUNCT punct
В ADP case
идеале NOUN ROOT
— PUNCT punct
добавить VERB parataxis
генерала 

### DeepPavlov

In [97]:
!pip install pymorphy2==0.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 426.0 kB/s eta 0:00:00 0:00:01
  Attempting uninstall: pymorphy2
    Found existing installation: pymorphy2 0.8
    Uninstalling pymorphy2-0.8:
      Successfully uninstalled pymorphy2-0.8


In [98]:
# !pip uninstall -y tensorflow tensorflow-gpu
!pip install numpy scipy librosa unidecode inflect librosa transformers
!pip install deeppavlov

  Using cached pydantic-1.10.2-cp38-cp38-macosx_10_9_x86_64.whl (3.1 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.3
    Uninstalling pydantic-1.3:
      Successfully uninstalled pydantic-1.3
  Using cached deeppavlov-0.17.6-py3-none-any.whl (878 kB)
  Using cached scikit-learn-0.21.2.tar.gz (12.2 MB)
  Preparing metadata (setup.py) ... done
  Using cached pymorphy2-0.8-py2.py3-none-any.whl (46 kB)
  Using cached pyTelegramBotAPI-3.6.7-py3-none-any.whl
  Using cached pydantic-1.3-py36.py37.py38-none-any.whl (85 kB)
  Using cached aio_pika-6.4.1-py3-none-any.whl (40 kB)
  Using cached aiormq-3.3.1-py3-none-any.whl (28 kB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [66 lines of output]
      Partial import of sklearn during the build process.
      blas_opt_info:
      blas_mkl_info:
      customize UnixCCompiler
        FOUND:
          libraries = ['mkl_rt', 'pthread']
    

**В этот блокнот не удалось установить библиотеку deeppalov из-за конфликта зависимостей.**

**Написать свой NER**

In [105]:
from sklearn.metrics import confusion_matrix, classification_report,f1_score, accuracy_score

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from razdel import tokenize
from corus import load_ne5

In [106]:
def get_classification_report(y_test_true, y_test_pred):
    print(classification_report(y_test_true, y_test_pred))

    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

Воспользуемся размеченным корпусом текстов.

In [111]:
records = load_ne5('Collection5/')
next(records)

Ne5Markup(
    id='1047',
    text='Жириновский предлагает обменять с США Сноудена на Бута\r\n\r\nЛидер ЛДПР Владимир Жириновский предложил обменять бывшего сотрудника ЦРУ США Эдварда Сноудена, который прибыл в Москву, на осужденного в Америке бизнесмена Виктора Бута.\r\n\r\n"Сноудена ни в коем случае не высылать в США, а обменять на Виктора Бута и Константина Ярошенко. В идеале — добавить генерала Олега Калугина", — написал он в своем микроблоге в Twitter.\r\n\r\nСноуден, работавший на компанию Booz Allen Hamilton — подрядчика Центрального разведывательного управления США, в начале июня распространил секретный ордер суда, по которому спецслужбы получили доступ ко всем звонкам крупнейшего сотового оператора Verizon, а также данные о сверхсекретной программе агентства национальной безопасности PRISM, позволяющей отслеживать электронные коммуникации на крупнейших сайтах. В воскресенье стало известно, что Сноуден прибыл из Гонконга в Москву и запросил убежища в Эквадоре.\r\n\r\nЧто ждет Э

In [112]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [113]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [114]:
df_words['tag'].value_counts()

OUT         219014
PER          21178
ORG          13641
LOC           4564
GEOPOLIT      4349
MEDIA         2481
Name: tag, dtype: int64

In [117]:
df_words.head()

,word,tag
0,Д,PER
1,.,PER
2,Медведев,PER
3,назначил,OUT
4,ряд,OUT


In [119]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

**Закодируем целевую переменную**

In [120]:
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

Посмотрим на классы

In [121]:
encoder.classes_

array(['GEOPOLIT', 'LOC', 'MEDIA', 'ORG', 'OUT', 'PER'], dtype=object)

In [122]:
train_x.apply(len).max(axis=0)

55

In [123]:
valid_x

234997        прошедшие
70628          назначен
47866                 в
191428          Василий
255655      возвращения
              ...      
184632                ,
94564       организаций
122979    парламентария
13008                на
163487          желанию
Name: word, Length: 66307, dtype: object

In [124]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

2022-10-25 21:18:31.866202: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [125]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [127]:
def custom_standardization(input_data):
    # Здесь может быть предобработка текста
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization(  
                            standardize=custom_standardization,
                            max_tokens=vocab_size,
                            output_mode='int',
                            #ngrams=(1, 3),
                            output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [128]:
len(vectorize_layer.get_vocabulary())

29842

In [134]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax') # [OUT, PER, ORG, LOC, GEOPOLIT, MEDIA]

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [135]:
mmodel = modelNER()

In [136]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [141]:
mmodel.fit( train_data,
            validation_data=valid_data,
            epochs=3)

Epoch 1/3
12433/12433 [==============================] - 243s 20ms/step - loss: 0.1452 - accuracy: 0.9571 - val_loss: 0.2059 - val_accuracy: 0.9395
Epoch 2/3
12433/12433 [==============================] - 255s 20ms/step - loss: 0.1124 - accuracy: 0.9653 - val_loss: 0.4621 - val_accuracy: 0.8940
Epoch 3/3
12433/12433 [==============================] - 281s 23ms/step - loss: 0.1055 - accuracy: 0.9666 - val_loss: 0.3598 - val_accuracy: 0.8942


In [142]:
pred_y = mmodel.predict(valid_x)
y_pred_classes = np.argmax(pred_y,axis=1)

2073/2073 [==============================] - 2s 1ms/step


In [143]:
f1 = f1_score(valid_y, y_pred_classes, average= "weighted")
f1

0.9010966691883908

In [144]:
print(f"Classes: {encoder.classes_}\r\n")

get_classification_report(valid_y, y_pred_classes)

Classes: ['GEOPOLIT' 'LOC' 'MEDIA' 'ORG' 'OUT' 'PER']

              precision    recall  f1-score   support

           0       0.88      0.90      0.89      1053
           1       0.85      0.78      0.81      1118
           2       0.95      0.77      0.85       631
           3       0.89      0.57      0.70      3424
           4       0.97      0.92      0.94     54777
           5       0.50      0.87      0.63      5304

    accuracy                           0.89     66307
   macro avg       0.84      0.80      0.80     66307
weighted avg       0.92      0.89      0.90     66307

CONFUSION MATRIX

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/mac/miniconda3/envs/pytorch_p38/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/r4/tj59nbc57_3bl6zqxrkxfk140000gn/T/ipykernel_2717/4235935365.py", line 3, in <cell line: 3>
    get_classification_report(valid_y, y_pred_classes)
  File "/var/folders/r4/tj59nbc57_3bl6zqxrkxfk140000gn/T/ipykernel_2717/1749492431.py", line 5, in get_classification_report
    print(pd.crosstab(y_test_true, y_test_pred))
  File "/Users/mac/miniconda3/envs/pytorch_p38/lib/python3.8/site-packages/pandas/core/reshape/pivot.py", line 679, in crosstab
    if len(names) != len(arrs):
  File "/Users/mac/miniconda3/envs/pytorch_p38/lib/python3.8/site-packages/pandas/core/frame.py", line 8721, in pivot_table
  File "/Users/mac/miniconda3/envs/pytorch_p38/lib/python3.8/site-packages/pandas/core/reshape/pivot.py", line 96, in pivot_table
    agged = grouped.a

#### Обучим нейронную сеть на биграммах и триграммах.

In [145]:
def custom_standardization(input_data):
    # Здесь может быть предобработка текста.
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization( 
                            standardize=custom_standardization,
                            max_tokens=vocab_size,
                            output_mode='int',
                            ngrams=(1, 3),
                            output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [146]:
mmodel = modelNER()

In [147]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [148]:
mmodel.fit( train_data,
            validation_data=valid_data,
            epochs=3)

Epoch 1/3
12433/12433 [==============================] - 262s 21ms/step - loss: 0.2946 - accuracy: 0.9144 - val_loss: 0.2097 - val_accuracy: 0.9378
Epoch 2/3
12433/12433 [==============================] - 484s 39ms/step - loss: 0.1258 - accuracy: 0.9624 - val_loss: 0.2078 - val_accuracy: 0.9406
Epoch 3/3
12433/12433 [==============================] - 354s 28ms/step - loss: 0.1091 - accuracy: 0.9659 - val_loss: 0.2096 - val_accuracy: 0.9408


In [149]:
pred_y = mmodel.predict(valid_x)
y_pred_classes = np.argmax(pred_y,axis=1)

2073/2073 [==============================] - 2s 1ms/step


In [150]:
f1 = f1_score(valid_y, y_pred_classes, average= "weighted")
f1

0.9363363752557727

In [151]:
print(f"Classes: {encoder.classes_}\r\n")

get_classification_report(valid_y, y_pred_classes)

Classes: ['GEOPOLIT' 'LOC' 'MEDIA' 'ORG' 'OUT' 'PER']

              precision    recall  f1-score   support

           0       0.87      0.92      0.89      1053
           1       0.86      0.76      0.81      1118
           2       0.95      0.77      0.85       631
           3       0.88      0.57      0.69      3424
           4       0.94      0.99      0.97     54777
           5       0.98      0.70      0.82      5304

    accuracy                           0.94     66307
   macro avg       0.91      0.78      0.84     66307
weighted avg       0.94      0.94      0.94     66307

CONFUSION MATRIX

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/mac/miniconda3/envs/pytorch_p38/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/r4/tj59nbc57_3bl6zqxrkxfk140000gn/T/ipykernel_2717/4235935365.py", line 3, in <cell line: 3>
    get_classification_report(valid_y, y_pred_classes)
  File "/var/folders/r4/tj59nbc57_3bl6zqxrkxfk140000gn/T/ipykernel_2717/1749492431.py", line 5, in get_classification_report
    print(pd.crosstab(y_test_true, y_test_pred))
  File "/Users/mac/miniconda3/envs/pytorch_p38/lib/python3.8/site-packages/pandas/core/reshape/pivot.py", line 679, in crosstab
    if len(names) != len(arrs):
  File "/Users/mac/miniconda3/envs/pytorch_p38/lib/python3.8/site-packages/pandas/core/frame.py", line 8721, in pivot_table
  File "/Users/mac/miniconda3/envs/pytorch_p38/lib/python3.8/site-packages/pandas/core/reshape/pivot.py", line 96, in pivot_table
    agged = grouped.a

### Результьат выше у сети, которая обучалась на N-граммах.